In [46]:
import pandas as pd
import numpy as np
from thefuzz import fuzz
from thefuzz import process

In [47]:
df_canal = pd.read_csv('./Datasets/CanalDeVenta.csv')
df_clientes = pd.read_csv('./Datasets/Clientes.csv', delimiter=';', encoding='utf-8')
df_compra = pd.read_csv('./Datasets/Compra.csv')
df_gasto = pd.read_csv('./Datasets/Gasto.csv')
df_local = pd.read_csv('./Datasets/Localidades.csv')
df_prov = pd.read_csv('./Datasets/Proveedores.csv', encoding='iso-8859-1')
df_suc = pd.read_csv('./Datasets/Sucursales.csv', delimiter=';')
df_tipo = pd.read_csv('./Datasets/TiposDeGasto.csv')
df_venta = pd.read_csv('./Datasets/Venta.csv')

# Tabla Venta

In [48]:
def get_moda(df, element_id, referencia, target):
    """Obtiene la moda de una columna target filtrando a través de la columna referencia"""
    df_moda = df[df[referencia] == element_id].mode()
    return df_moda[target][0]

In [49]:
def input_moda(df, element_id, referencia, target):
    """Cambia todos los valores de las columnas al valor de la moda"""
    moda = get_moda(df, element_id, referencia, target)
    df.loc[(df[target] != moda) & (df[referencia] == element_id), target] = moda

In [50]:
def fill_moda(df, columna_filtro, target):
    productos = [x for x in df[columna_filtro].unique()]
    for product in productos:
        input_moda(df_venta, product, columna_filtro, target)

In [51]:
fill_moda(df_venta, 'IdProducto', 'Precio')

# Localidad

In [52]:
df_provincias = df_local.loc[:,['provincia_id','provincia_nombre']].drop_duplicates()
df_localidades = df_local.loc[:,['nombre', 'id']].drop_duplicates()

# Tabla Clientes

In [53]:
df_clientes = df_clientes.drop(columns='col10')

In [54]:
def get_apellidos(df, columna):
    apellidos = []
    for nombre in df[columna]:
        if isinstance(nombre, str):
            nombre = nombre.upper()
            if ', ' in nombre:
                apellidos.append(nombre.split(', ')[0])
            else:
                words = nombre.split(' ')
                if len(words) > 3:
                    last = words[-2:]
                    apellidos.append(' '.join(last))
                else:
                    apellidos.append(words[-1])
        else:
            apellidos.append(float('NaN'))
    return apellidos

In [55]:
def get_names(element):
    """Obtengo sólo los nombres"""
    if isinstance(element, str):
        element = element.upper()
        if ', ' in element:
            return element.split(', ')[1]

        names_last = element.split(' ')
        if len(names_last) > 3:
            names = names_last[:-2]
        else:
            names = names_last[:-1]

        return ' '.join(names)

In [56]:
def nombre_apellido(df):
    """Creo una columna de apellidos y nombres por separado"""
    apellidos = get_apellidos(df, 'Nombre_y_Apellido')
    df.insert(3, 'Apellido', apellidos)
    df['Nombre_y_Apellido'] = df['Nombre_y_Apellido'].apply(get_names)
    df.rename(columns={'Nombre_y_Apellido':'Nombre'}, inplace=True)


In [57]:
nombre_apellido(df_clientes)

In [58]:
def provincia_to_id(element, target):
    """Obtengo el Id de la provincia y reemplazo su nombre"""
    if target == 'Provincia':
        search = 'provincia_id'
        filter = 'provincia_nombre'
        df = df_provincias
    elif target == 'Localidad':
        search = 'id'
        filter = 'nombre'
        df = df_localidades
    else:
        return float('NaN')
    if isinstance(element, str):
        mask = df[df[filter].apply(lambda x: fuzz.partial_ratio(x,element)) > 80] # Aplico la función de fuzz, que implementa Levinshtein, para obtener el id de la provincia
        return mask[search].values.tolist()[0]
    else:
        return element

In [59]:
def idprovincia(df, target):
    """Aplica la función provincia_to_id a un dataset"""
    df[target] = df[target].apply(lambda x: provincia_to_id(x, target))
    df.rename(columns={target:f'Id{target}'}, inplace=True)
    df[f'Id{target}'] = df[f'Id{target}'].astype('Int64')

In [60]:
idprovincia(df_clientes, 'Provincia')

In [62]:
def replace_comma(x):
    if isinstance(x, str) and len(x) > 2:
        return float(x.replace(',', '.'))
    else:
        return x

In [63]:
def input_replaced_comma(df):
    """Se aplica la función replace_comma a un dataset"""
    df.X = df.X.apply(replace_comma)
    df.Y = df.Y.apply(replace_comma)

In [64]:
input_replaced_comma(df_clientes)

In [66]:
def closer_location(df, cen_lon, cen_lat, lon, lat, target):
    """De acuerdo a las coordenadas del cliente, determino cuál es la localización más cercana"""
    result = np.nan
    for i in range(1,500): # Itero para varios radios
        if len(df[(df[cen_lon].apply(lambda x: int(abs(x-lon)*i)) == 0) & (df[cen_lat].apply(lambda y: int(abs(y-lat)*i)) == 0)]) == 0:
            result = df[(df[cen_lon].apply(lambda x: int(abs(x-lon)*(i-1))) == 0) & (df[cen_lat].apply(lambda y: int(abs(y-lat)*(i-1))) == 0)]
            break
    return result[target].values.tolist()[0]

In [67]:
def get_location(x, df, target_col):
    """Lleno los valores faltantes en localidad o provincia usando las coordenadas"""
    na_local = df[df[target_col].isna()]
    if x.name == target_col:
        names = []
        ind = x.index
        if target_col == 'IdProvincia':
            target = 'provincia_id'
        else:
            target = 'nombre'
        lon = na_local.loc[ind, 'X'].values
        lat = na_local.loc[ind, 'Y'].values
        for i in range(len(lon)):
            names.append(closer_location(df_local, 'centroide_lon', 'centroide_lat', lon[i], lat[i], target))
        return pd.Series(names, ind)
    else:
        return x


In [68]:
def fill_location(df, target):
    df[df[target].isna()] = df[df[target].isna()].apply(lambda x: get_location(x, df, target))

In [69]:
fill_location(df_clientes, 'Localidad')
fill_location(df_clientes, 'IdProvincia')

In [70]:
df_clientes.fillna('None', inplace=True)

# Tabla Compra

In [71]:
def fill_precio(x):
    """Completa los precios faltantes de acuerdo a la media de cada producto"""
    means_product_prices = df_compra.groupby(by='IdProducto').mean().loc[:,'Precio']
    id_prod = df_compra[df_compra['Precio'].isna()]['IdProducto']
    if x.name == 'Precio':
        price = means_product_prices[id_prod]
        return np.round(price.values.tolist(), 2)
    else:
        return x

In [72]:
df_compra[df_compra['Precio'].isna()] = df_compra[df_compra['Precio'].isna()].apply(fill_precio)

# Tabla Sucursales

In [73]:
def fix_provincia(x, target):
    """Corrijo la provincia o localidad de una sucursal"""
    if isinstance(x, str):
        result = process.extractOne(x.lower(), ['ciudad de buenos aires', 'caba', 'bs as', 'capital federal', 'cap fed'],score_cutoff=80)
        if result and result[1] > 80 and target == 'Provincia':
            text = 'BUENOS AIRES'
        elif result and result[1] > 80 and target == 'Localidad':
            text = 'CIUDAD DE BUENOS AIRES'
        else:
            text = x.upper()
        return text
    else:
        return x

In [74]:
df_suc['Provincia'] = df_suc['Provincia'].apply(lambda x: fix_provincia(x, 'Provincia'))
df_suc['Localidad'] = df_suc['Localidad'].apply(lambda x: fix_provincia(x, 'Localidad'))

# Tabla Proveedores

In [75]:
df_prov.rename(columns={'Address':'Direccion', 'City':'Localidad', 'State': 'Provincia'}, inplace=True)
df_prov.drop(columns=['departamen', 'Country'], inplace=True)

In [76]:
df_prov['Localidad'][df_prov['Provincia'] == 'CABA'] = 'CIUDAD DE BUENOS AIRES'
df_prov['Provincia'] = df_prov['Provincia'].apply(lambda x: fix_provincia(x, 'Provincia'))

<ipython-input-76-679092b53d29>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prov['Localidad'][df_prov['Provincia'] == 'CABA'] = 'CIUDAD DE BUENOS AIRES'


In [78]:
def fill_precio(x):
    """Completa los precios faltantes de acuerdo a la media de cada producto"""
    means_product_prices = df_compra.groupby(by='IdProducto').mean().loc[:,'Precio']
    id_prod = df_compra[df_compra['Precio'].isna()]['IdProducto']
    if x.name == 'Precio':
        price = means_product_prices[id_prod]
        return np.round(price.values.tolist(), 2)
    else:
        return x

In [79]:
df_compra[df_compra['Precio'].isna()] = df_compra[df_compra['Precio'].isna()].apply(fill_precio)

# Tabla Sucursales

In [80]:
df_suc

,ID,Sucursal,Direccion,Localidad,Provincia,Latitud,Longitud
0,1,Cabildo,Av. Cabildo 1342,CIUDAD DE BUENOS AIRES,BUENOS AIRES,"-34,5678060","-58,4495720"
1,2,Palermo 1,Guatemala 5701,CIUDAD DE BUENOS AIRES,BUENOS AIRES,"-34,5790350","-58,4335660"
2,3,Palermo 2,Gral. Lucio Norberto Mansilla 2668,CIUDAD DE BUENOS AIRES,BUENOS AIRES,"-34,5959660","-58,4051500"
3,4,Corrientes,Av. Corrientes 2352,CIUDAD DE BUENOS AIRES,BUENOS AIRES,"-34,6046850","-58,3987640"
4,5,Almagro,Venezuela 3650,CIUDAD DE BUENOS AIRES,BUENOS AIRES,"-34,6173080","-58,4161790"
5,6,Caballito,Av. Rivadavia 4708,CIUDAD DE BUENOS AIRES,BUENOS AIRES,"-34,6163030","-58,4318490"
6,7,Flores,Av. Rivadavia 5746,CIUDAD DE BUENOS AIRES,BUENOS AIRES,"-34,6228680","-58,4464490"
7,8,Alberdi,Av. Juan Bautista Alberdi 1634,CIUDAD DE BUENOS AIRES,BUENOS AIRES,"-34,6273060","-58,4514980"
8,9,Deposito,Pedernera 530,CIUDAD DE BUENOS AIRES,BUENOS AIRES,"-34,6351340","-58,4615130"
9,10,Velez,Av. Juan Bautista Justo 7738,CIUDAD DE BUENOS AIRES,BUENOS AIRES,"-34,6312570","-58,4980630"
